# Description

It standardize (z-score) S-MultiXcan results projected into the MultiPLIER latent space.

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from IPython.display import display

import numpy as np
import pandas as pd
from sklearn.preprocessing import scale

import conf

# Settings

In [3]:
np.random.seed(0)

In [4]:
NULL_DIR = conf.RESULTS["CLUSTERING_NULL_DIR"] / "shuffle_genes"

## Input data

In [5]:
INPUT_FILEPATH = Path(
    NULL_DIR,
    "projections",
    "projection-smultixcan-efo_partial-mashr-zscores.pkl",
).resolve()
display(INPUT_FILEPATH)

input_filepath_stem = INPUT_FILEPATH.stem
display(input_filepath_stem)

PosixPath('/opt/data/results/clustering/null_sims/shuffle_genes/projections/projection-smultixcan-efo_partial-mashr-zscores.pkl')

'projection-smultixcan-efo_partial-mashr-zscores'

## Output folder

In [6]:
# output dir for this notebook
RESULTS_DIR = Path(NULL_DIR, "data_transformations", "z_score_std").resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_DIR)

PosixPath('/opt/data/results/clustering/null_sims/shuffle_genes/data_transformations/z_score_std')

# Load input file

In [7]:
data = pd.read_pickle(INPUT_FILEPATH).T

In [8]:
display(data.shape)

(3752, 987)

In [9]:
display(data.head())

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
100001_raw-Food_weight,-0.004991,-0.020160,-0.026030,0.020068,0.011753,0.017012,0.049228,-0.027804,0.020122,-0.043951,...,-0.027612,0.011046,-0.014123,-0.026495,-0.034632,-0.007654,0.039811,0.001866,-0.026045,0.024954
100002_raw-Energy,0.015146,0.025381,0.030990,-0.004168,0.006254,0.014020,0.006114,-0.039952,-0.004144,0.029471,...,-0.017800,0.018595,-0.003060,-0.044417,0.021673,0.009405,-0.019020,0.042079,0.016351,0.032201
100003_raw-Protein,0.011558,0.026267,-0.008830,0.023151,-0.014700,0.015522,0.010072,0.005893,0.030038,-0.053161,...,-0.014480,0.008231,-0.004317,-0.038221,0.008717,0.017687,0.014660,-0.003575,-0.024418,-0.032628
100004_raw-Fat,-0.004454,0.025022,0.010287,-0.045471,0.035624,0.054628,-0.015242,-0.031907,0.007947,0.024257,...,-0.012052,-0.036220,-0.009773,0.024453,-0.020112,0.032634,-0.000500,0.010128,-0.000518,0.051511
100005_raw-Carbohydrate,-0.004064,-0.014816,0.027811,0.015872,-0.012119,-0.035210,-0.035111,0.026068,0.031480,0.001849,...,0.044601,-0.002078,-0.002292,-0.038174,-0.012224,0.012797,0.048189,-0.026752,0.000800,0.019285


# z-score standardization

In [10]:
data_stats = data.iloc[:, :10].describe()
display(data_stats)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10
count,3.752000e+03,3.752000e+03,3.752000e+03,3.752000e+03,3.752000e+03,3.752000e+03,3.752000e+03,3.752000e+03,3.752000e+03,3.752000e+03
mean,4.260984e-18,2.130492e-18,-1.006066e-18,2.308033e-18,-3.550820e-19,-1.242787e-18,4.231394e-18,3.402869e-18,-4.438525e-19,-2.441189e-18
std,2.851547e-02,2.839185e-02,2.601169e-02,2.896133e-02,2.871084e-02,2.646271e-02,2.785000e-02,2.597797e-02,2.486356e-02,2.826685e-02
min,-1.120355e-01,-1.015075e-01,-8.524020e-02,-1.071991e-01,-8.668615e-02,-1.215630e-01,-9.090851e-02,-8.776256e-02,-9.062002e-02,-8.284056e-02
25%,-1.855408e-02,-1.926075e-02,-1.722247e-02,-1.968283e-02,-2.006003e-02,-1.810400e-02,-1.863630e-02,-1.769060e-02,-1.649458e-02,-1.952048e-02
50%,-8.137997e-04,-8.927103e-04,-1.458560e-04,-1.200660e-04,-1.103916e-03,1.469371e-04,-1.021344e-03,-9.058482e-04,-7.845392e-05,-2.889262e-04
75%,1.803097e-02,1.816413e-02,1.684615e-02,1.861076e-02,1.829824e-02,1.733025e-02,1.766101e-02,1.689003e-02,1.608497e-02,1.776515e-02
max,1.318461e-01,1.379319e-01,2.187188e-01,1.073417e-01,1.785174e-01,1.469357e-01,1.004747e-01,1.862624e-01,1.982892e-01,1.460073e-01


In [11]:
scaled_data = pd.DataFrame(
    data=scale(data), index=data.index.copy(), columns=data.columns.copy()
)

In [12]:
display(scaled_data.shape)

(3752, 987)

In [13]:
display(scaled_data.head())

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
100001_raw-Food_weight,-0.175043,-0.710171,-1.000846,0.693026,0.409413,0.642958,1.767862,-1.070425,0.809395,-1.555050,...,-0.932802,0.371349,-0.521787,-0.881136,-1.193309,-0.257883,1.382268,0.066549,-0.866086,0.847997
100002_raw-Energy,0.531227,0.894062,1.191552,-0.143922,0.217867,0.529878,0.219561,-1.538143,-0.166704,1.042749,...,-0.601329,0.625139,-0.113048,-1.477163,0.746773,0.316892,-0.660407,1.501067,0.543744,1.094294
100003_raw-Protein,0.405395,0.925288,-0.339507,0.799476,-0.512075,0.586632,0.361706,0.226860,1.208289,-1.880931,...,-0.489171,0.276717,-0.159505,-1.271106,0.300351,0.595915,0.509020,-0.127540,-0.811987,-1.108812
100004_raw-Fat,-0.156230,0.881426,0.395527,-1.570256,1.240955,2.064595,-0.547373,-1.228389,0.319684,0.858253,...,-0.407151,-1.217662,-0.361052,0.813221,-0.692981,1.099523,-0.017366,0.361278,-0.017222,1.750511
100005_raw-Carbohydrate,-0.142531,-0.521907,1.069304,0.548122,-0.422146,-1.330718,-1.260872,1.003617,1.266289,0.065408,...,1.506698,-0.069854,-0.084662,-1.269536,-0.421185,0.431180,1.673162,-0.954336,0.026593,0.655359


In [14]:
scaled_data_stats = scaled_data.iloc[:, :10].describe()
display(scaled_data_stats)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10
count,3.752000e+03,3.752000e+03,3.752000e+03,3.752000e+03,3.752000e+03,3.752000e+03,3.752000e+03,3.752000e+03,3.752000e+03,3.752000e+03
mean,1.893771e-18,1.041574e-17,-1.515016e-17,2.272525e-17,-1.041574e-17,-1.893771e-17,-8.995410e-18,2.177836e-17,-1.420328e-17,-1.230951e-17
std,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00
min,-3.929463e+00,-3.575712e+00,-3.277433e+00,-3.701949e+00,-3.019686e+00,-4.594361e+00,-3.264654e+00,-3.378796e+00,-3.645178e+00,-2.931052e+00
25%,-6.507538e-01,-6.784805e-01,-6.621935e-01,-6.797152e-01,-6.987850e-01,-6.842237e-01,-6.692562e-01,-6.810757e-01,-6.634920e-01,-6.906704e-01
50%,-2.854269e-02,-3.144668e-02,-5.608072e-03,-4.146288e-03,-3.845457e-02,5.553349e-03,-3.667792e-02,-3.487451e-02,-3.155798e-03,-1.022274e-02
75%,6.324069e-01,6.398510e-01,6.477239e-01,6.426929e-01,6.374135e-01,6.549806e-01,6.342319e-01,6.502541e-01,6.470156e-01,6.285638e-01
max,4.624285e+00,4.858798e+00,8.409602e+00,3.706873e+00,6.218598e+00,5.553296e+00,3.608190e+00,7.170969e+00,7.976157e+00,5.166008e+00


## Testing

In [15]:
assert np.all(
    [
        np.isclose(scaled_data_stats.loc["mean", c], 0.0)
        for c in scaled_data_stats.columns
    ]
)

In [16]:
assert np.all(
    [
        np.isclose(scaled_data_stats.loc["std", c], 1.0, atol=1e-03)
        for c in scaled_data_stats.columns
    ]
)

# Save

In [17]:
output_file = Path(
    RESULTS_DIR,
    f"z_score_std-{input_filepath_stem}.pkl",
).resolve()

display(output_file)

PosixPath('/opt/data/results/clustering/null_sims/shuffle_genes/data_transformations/z_score_std/z_score_std-projection-smultixcan-efo_partial-mashr-zscores.pkl')

In [18]:
scaled_data.to_pickle(output_file)